In [ ]:
# Confocal logic

In [ ]:
    def _scan_line(self):
        """scanning an image in either depth or xy

        """
        image = self.depth_image if self._zscan else self.xy_image
        n_ch = len(self.get_scanner_axes())
        s_ch = len(self.get_scanner_count_channels())


        # stops scanning
        if self.stopRequested:
            with self.threadlock:
                self.stopRequested = False
                self.module_state.unlock()
                self.signal_xy_image_updated.emit()
                self.signal_depth_image_updated.emit()

                # Used to be like this
                # # FIXME RETURN IT BACK NORMALLY!
                # self.set_position('scanner')

                # Javid changed to this (29.02.2020)
                # self._current_x, self._current_y, self._current_z, self._current_a

                rs = self.return_slowness

                lsx = np.linspace(image[self._scan_counter-1,-1,0],self._current_x, rs)
                lsy = np.linspace(image[self._scan_counter-1,-1,1],self._current_y,  rs)
                lsz = np.linspace(image[self._scan_counter-1,-1,2],self._current_z,  rs)




                if n_ch <= 3:
                    end_line = np.vstack([lsx, lsy, lsz][0:n_ch])
                else:
                    end_line = np.vstack(
                        [lsx, lsy, lsz, np.ones(lsx.shape) * self._current_a])
                # move to the coursor position, counts are thrown away
                end_line_counts = self._scanning_device.scan_line(end_line)

                self.kill_scanner()

                if np.any(end_line_counts == -1):
                    self.log.exception('Something wrong with end_line_counts')
                    return


                if self._zscan:
                    self._depth_line_pos = self._scan_counter
                else:
                    self._xy_line_pos = self._scan_counter
                # add new history entry
                new_history = ConfocalHistoryEntry(self)
                new_history.snapshot(self)
                self.history.append(new_history)
                if len(self.history) > self.max_history_length:
                    self.history.pop(0)
                self.history_index = len(self.history) - 1
                return


In [ ]:
# optimizer logic

In [ ]:
 def finish_refocus(self):
        """ Finishes up and releases hardware after the optimizer scans."""

        image = self.xy_refocus_image
        n_ch = len(self._scanning_device.get_scanner_axes())

        # s_ch = len(self.get_scanner_count_channels())
        #
        rs = self.return_slowness
        #
        lsx = np.linspace(image[self._xy_scan_line_count - 1, -1, 0], self.optim_pos_x, rs)
        lsy = np.linspace(image[self._xy_scan_line_count - 1, -1, 1],self.optim_pos_y,  rs)
        lsz = np.linspace(image[self._xy_scan_line_count - 1, -1, 2],self.optim_pos_z,  rs)
        #
        if n_ch <= 3:
            end_line = np.vstack([lsx, lsy, lsz][0:n_ch])
        else:
            end_line = np.vstack(
                [lsx, lsy, lsz, np.ones(lsx.shape) * self._current_a])
        # # move to the coursor position, counts are thrown away
        # print('self.optim_pos_x ',self.optim_pos_x)
        end_line_counts = self._scanning_device.scan_line(end_line)
        # print('self._scanning_device. in opt log ',self._scanning_device.get_scanner_position())

        print('===============')

        self.kill_scanner()

        self.log.info(
                'Optimised from ({0:.3e},{1:.3e},{2:.3e}) to local '
                'maximum at ({3:.3e},{4:.3e},{5:.3e}).'.format(
                    self._initial_pos_x,
                    self._initial_pos_y,
                    self._initial_pos_z,
                    self.optim_pos_x,
                    self.optim_pos_y,
                    self.optim_pos_z))

        # Signal that the optimization has finished, and "return" the optimal position along with
        # caller_tag
        self.sigRefocusFinished.emit(
            self._caller_tag,
            [self.optim_pos_x, self.optim_pos_y, self.optim_pos_z, 0])


In [4]:
optimizer._x_shift

NameError: name 'optimizer' is not defined

In [1]:
7

7